In [1]:
import numpy as np
from tensorboard_evaluation import *

In [2]:
x = np.load("replay_buffer/data.npz")

In [3]:
x.__dict__

{'_files': ['state.npy',
  'action.npy',
  'next_state.npy',
  'reward.npy',
  'not_done.npy'],
 'files': ['state', 'action', 'next_state', 'reward', 'not_done'],
 'allow_pickle': False,
 'max_header_size': 10000,
 'pickle_kwargs': {'encoding': 'ASCII', 'fix_imports': True},
 'zip': <zipfile.ZipFile file=<_io.BufferedReader name='replay_buffer/data.npz'> mode='r'>,
 'f': <numpy.lib.npyio.BagObj at 0x7f522fa47e80>,
 'fid': <_io.BufferedReader name='replay_buffer/data.npz'>}

In [4]:
state = x["state"]
next_state = x["next_state"]
reward = x["reward"]
not_done = x["not_done"]
action = x["action"]

In [5]:
state.shape

(59000, 23)

In [6]:
import numpy as np
import torch


class ReplayBuffer(object):
	def __init__(self, state_dim, action_dim, max_size=int(1e6)):
		self.max_size = max_size
		self.ptr = 59000
		self.size = 59000

		self.state = state
		self.action = action
		self.next_state = next_state
		self.reward = reward
		self.not_done = not_done

		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


	def add(self, state, action, next_state, reward, done):
		self.state[self.ptr] = state
		self.action[self.ptr] = action
		self.next_state[self.ptr] = next_state
		self.reward[self.ptr] = reward
		self.not_done[self.ptr] = 1. - done

		self.ptr = (self.ptr + 1) % self.max_size
		self.size = min(self.size + 1, self.max_size)


	def sample(self, batch_size):
		ind = np.random.randint(0, self.size, size=batch_size)

		return (
			torch.FloatTensor(self.state[ind]).to(self.device),
			torch.FloatTensor(self.action[ind]).to(self.device),
			torch.FloatTensor(self.next_state[ind]).to(self.device),
			torch.FloatTensor(self.reward[ind]).to(self.device),
			torch.FloatTensor(self.not_done[ind]).to(self.device)
		)
	
	def save(self,filename):
		np.savez(filename,state = self.state[:self.size,::],action = self.action[:self.size,::],\
			next_state = self.next_state[:self.size,::],reward = self.reward[:self.size,::],not_done =self.not_done[:self.size,::])
	

In [7]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_agent.agent_builder_ddpg_hit import build_agent
# from air_hockey_challenge.environments.planar.hit import AirHockeyHit
from tensorboard_evaluation import *

pybullet build time: May 20 2022 19:44:17


In [8]:
env = AirHockeyChallengeWrapper(env="7dof-hit", interpolation_order=3, debug=False)
# policy = build_agent(env.env_info)
policy = build_agent(env.env_info) 

In [9]:
replay_buffer = ReplayBuffer(23, 14)

In [10]:
main_dir = "/run/media/luke/Data/uni/SS2023/DL Lab/Project/qualifying/DDPG"
tensorboard_dir=main_dir + "/tensorboard/"
tensorboard = Evaluation(tensorboard_dir, "train", ["critic_loss","actor_loss"])


In [11]:
for epoch in range(1500):
    critic_loss=np.nan
    actor_loss=np.nan
    critic_loss,actor_loss = policy.train(replay_buffer,batch_size= 512)
    tensorboard.write_episode_data(epoch, eval_dict={ "critic_loss" : critic_loss,\
                "actor_loss":actor_loss})

In [13]:
policy.save(main_dir + f"/models/DDPG-v0_air-hockey_0")